# Schema Translation with AgentCore Gateway Interceptor 

### Overview

This notebook shows how to use AgentCore Gateway for **Schema Translation** between existing service contracts and MCP tool schemas, so agents can call your APIs without any custom protocol glue. The goal is to give you a reusable pattern to normalize heterogeneous backend schemas into a clean, agent-friendly interface, while keeping your source systems unchanged.

### Why Schema Translation Matters

When building AI agents that interact with multiple backend services, schema inconsistencies can create integration complexity and slow down agent development. Schema translation via the AgentCore Gateway allows you to normalize and unify diverse API schemas into a consistent MCP format that your agents can easily consume. 

This approach:

- Eliminates the need to modify existing APIs or backends while exposing them as MCP-compatible tools
- Centralizes protocol translation and schema evolution, reducing maintenance overhead
- Enables seamless integration of REST APIs, Lambda functions, and MCP servers behind a single gateway
- Provides a scalable, maintainable way to keep your agent’s view of tools clean and consistent

Gateway interceptors provide a scalable, plug-and-play way to implement schema translation **without modifying the agent, the runtime, or the MCP server**.  
You enforce policy **at the Gateway level**, where every request naturally flows through.

---

### What This Tutorial Covers

You will implement Schema Translation by:

1. 🛠️ **Create AgentCore Gateway with Request Gateway Interceptor**:
The Request interceptor has logic to transform the schemas in the request and to add any additional parameters. 

2. 🔍 **Register tools with the AgentCore Gateway**: Register the inventory tool with the AgentCore Gateway 

3. 📋 **Invoke tool**: When the tool is invoked,the request is intercepted and the schema translation is performed (for example: item_id -> itemId or adding additional parameters to request such as location etc.)

![schema-translation](images/schema-translation-request-interceptor.png)

---

### Why Use Gateway Interceptors?

Gateway interceptors allow you to add logic for:

- **Performing Schema Translation**
Seamlessly convert request and response schemas between heterogeneous backend APIs and the MCP format.

- **Normalizing Tool Interfaces**
Create a consistent, agent-friendly schema abstraction across diverse services without changing underlying APIs.

- **Adapt Protocols and Data Formats**
Modify requests or responses to bridge differences in expected parameters, field names, or data structures.

- **Centralize Translation Logic**
Manage schema evolution and transformations in one place to simplify maintenance and improve scalability.

Because interceptors operate at the Gateway layer, they enable consistent schema handling for any connected MCP server or runtime without altering individual tool implementations.



---

### Tutorial Details

| Information              | Details                                                                                         |
|--------------------------|-------------------------------------------------------------------------------------------------|
| **Tutorial type**        | Interactive                                                                                     |
| **AgentCore components** | AgentCore Gateway, Gateway Interceptors (Request)                                               |
| **Gateway Target type**  | Amazon Bedrock AgentCore Gateway, Gateway Interceptors                      |
| **Gateway Target type**  | MCP Server (Lambda-based tool)                                                                                |
| **Interceptor types**    | AWS Lambda (REQUEST)                                                                            |
| **Inbound Auth IdP**     | Amazon Cognito (CUSTOM\_JWT authorizer)                                                         |
| **Tutorial components**  | Gateway, Cognito, Gateway Interceptors, MCP tools (lambda)                                      |
| **Tutorial vertical**    | Cross-vertical                                                                                  |
| **Example complexity**   | Intermediate                                                                               |
| **SDK used**             | boto3                                                                                           |

---

## Prerequisites

To execute this tutorial you will need:

- Jupyter notebook (Python kernel)
- AWS credentials with permissions for:
  - AWS Lambda
  - AWS IAM
  - Amazon Cognito
  - Amazon Bedrock AgentCore services (control plane)
- Python 3.9 or higher
- Basic understanding of AWS Lambda, IAM roles, Amazon Cognito, and Amazon Bedrock AgentCore Gateway

> ⚠️ **Note:** The Cleanup section at the end deletes the AWS resources created by this tutorial (Gateway, Lambdas, IAM roles, etc.). Only run it when you're ready to tear everything down.

---

## Part 1: Setup & Deployment

### Step 1.1: Import Required Libraries

In [13]:
import boto3
import json
import time
import sys
from pathlib import Path
from datetime import datetime

# Add parent directory to path for utils
utils_dir = Path.cwd().parent
sys.path.insert(0, str(utils_dir))

import utils

print("✓ Libraries imported")

# Generate unique identifier for this deployment
DEPLOYMENT_ID = datetime.now().strftime('%Y%m%d-%H%M%S')
print(f"\nDeployment ID: {DEPLOYMENT_ID}")

✓ Libraries imported

Deployment ID: 20251126-155213


### Step 1.2: Configure Deployment Variables

In [14]:
# Configuration
REGION = "us-east-1"  
LAMBDA_FUNCTION_NAME = f"interceptor-lambda-{DEPLOYMENT_ID}"
LAMBDA_ROLE_NAME = f"interceptor-lambda-role-{DEPLOYMENT_ID}"
GATEWAY_NAME = f"interceptor-gateway-{DEPLOYMENT_ID}"

# Initialize clients
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
cognito_client = boto3.client('cognito-idp', region_name=REGION)

print("Configuration:")
print(f"  Lambda Function: {LAMBDA_FUNCTION_NAME}")
print(f"  Lambda Role: {LAMBDA_ROLE_NAME}")
print(f"  Gateway Name: {GATEWAY_NAME}")
print(f"  Region: {REGION}")

Configuration:
  Lambda Function: interceptor-lambda-20251126-155213
  Lambda Role: interceptor-lambda-role-20251126-155213
  Gateway Name: interceptor-gateway-20251126-155213
  Region: us-east-1


### Step 1.3: Create IAM Role for Lambda Interceptor

Grant Lambda permissions to execute and write CloudWatch logs.

In [15]:
# Create IAM role for Lambda interceptor using utils
print("Creating IAM role for Lambda interceptor...")

LAMBDA_ROLE_ARN = utils.create_lambda_role(
    role_name=LAMBDA_ROLE_NAME,
    description='Role for AgentCore Lambda Interceptor for schema translation'
)

print(f"  ARN: {LAMBDA_ROLE_ARN}")

Creating IAM role for Lambda interceptor...
✓ IAM role created: interceptor-lambda-role-20251126-155213
  ARN: arn:aws:iam::101546054019:role/interceptor-lambda-role-20251126-155213


### Step 1.4: Deploy Lambda Interceptor Function

Lambda intercepts tool requests and translates schema parameters before forwarding to targets.

In [19]:
# Deploy Lambda interceptor using utils
print("Deploying Lambda interceptor...")

LAMBDA_ARN = utils.deploy_lambda_function(
    function_name=LAMBDA_FUNCTION_NAME,
    role_arn=LAMBDA_ROLE_ARN,
    lambda_code_path='src/lambda/lambda_function.py',
    description='AgentCore Request Lambda Interceptor for schema translation',
    timeout=30,
    memory_size=256,
    region=REGION
)

print(f"  ARN: {LAMBDA_ARN}")

Deploying Lambda interceptor...
⚠ Lambda already exists: interceptor-lambda-20251126-155213
  ARN: arn:aws:lambda:us-east-1:101546054019:function:interceptor-lambda-20251126-155213


In [20]:
# Grant Gateway permission to invoke the Lambda interceptor
print("\nGranting Gateway permission to invoke Lambda...")

utils.grant_gateway_invoke_permission(
    function_name=LAMBDA_FUNCTION_NAME,
    region=REGION
)


Granting Gateway permission to invoke Lambda...
⚠ Permission already exists (this is fine)


### Step 1.5: Create Amazon Cognito User Pool & App Client

Create Cognito user pool for Gateway authentication using OAuth client credentials flow.

In [18]:
# Create Cognito User Pool and Client for Gateway authentication using utils
print("Creating Cognito User Pool and Client...")

USER_POOL_NAME = f"gateway-pool-{DEPLOYMENT_ID}"
RESOURCE_SERVER_ID = 'gateway'
RESOURCE_SERVER_NAME = 'Gateway Resource Server'
SCOPES = [{'ScopeName': 'tools', 'ScopeDescription': 'Access to gateway tools'}]

# Create or get user pool
USER_POOL_ID = utils.get_or_create_user_pool(cognito_client, USER_POOL_NAME)
print(f"  Pool ID: {USER_POOL_ID}")

# Create or get resource server
utils.get_or_create_resource_server(cognito_client, USER_POOL_ID, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)

# Wait for resource server to propagate
print("  Waiting for resource server to propagate...")
time.sleep(3)

# Create M2M client with client credentials flow
CLIENT_NAME = f"gateway-client-{DEPLOYMENT_ID}"
CLIENT_ID, CLIENT_SECRET = utils.get_or_create_m2m_client(
    cognito_client,
    USER_POOL_ID,
    CLIENT_NAME,
    RESOURCE_SERVER_ID,
    SCOPES=[f"{RESOURCE_SERVER_ID}/tools"]
)

print(f"✓ User Pool Client created: {CLIENT_NAME}")
print(f"  Client ID: {CLIENT_ID}")
print(f"  Client Secret: {CLIENT_SECRET[:20]}...")

# Construct OAuth URLs
POOL_DOMAIN = USER_POOL_ID.replace('_', '').lower()
COGNITO_DOMAIN = f"https://{POOL_DOMAIN}.auth.{REGION}.amazoncognito.com"
DISCOVERY_URL = f"https://cognito-idp.{REGION}.amazonaws.com/{USER_POOL_ID}/.well-known/openid-configuration"
TOKEN_URL = f"{COGNITO_DOMAIN}/oauth2/token"

print(f"\n✓ OAuth Configuration:")
print(f"  Discovery URL: {DISCOVERY_URL}")
print(f"  Token URL: {TOKEN_URL}")
print(f"  Scope: {RESOURCE_SERVER_ID}/tools")

Creating Cognito User Pool and Client...
Creating new user pool
Domain created as well
  Pool ID: us-east-1_vk24EZrgb
creating new resource server
  Waiting for resource server to propagate...
creating new m2m client
✓ User Pool Client created: gateway-client-20251126-155213
  Client ID: 6ff968ucqm2f5a5rbqejg8pobg
  Client Secret: 1791ccrqi8jadocbd7pv...

✓ OAuth Configuration:
  Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_vk24EZrgb/.well-known/openid-configuration
  Token URL: https://us-east-1vk24ezrgb.auth.us-east-1.amazoncognito.com/oauth2/token
  Scope: gateway/tools


### Step 1.6: Create Gateway with Request Interceptor

**Why REQUEST Interceptor?**  
The interceptor processes tool requests before execution, allowing us to translate schema parameters.

In [21]:
# Create Gateway IAM role
gateway_iam_role = utils.create_agentcore_gateway_role_with_region(GATEWAY_NAME, REGION)
GATEWAY_ROLE_ARN = gateway_iam_role['Role']['Arn']

print(f"✓ Gateway role created: {GATEWAY_ROLE_ARN}")

# Wait for role propagation
time.sleep(10)

# Create Gateway with Lambda interceptor
print(f"\nCreating Gateway with RESPONSE interceptor...")

try:
    gateway_response = gateway_client.create_gateway(
        name=GATEWAY_NAME,
        protocolType="MCP",
        protocolConfiguration={
            "mcp": {
                "supportedVersions": ["2025-03-26"]
            }
        },
        interceptorConfigurations=[
            {
                "interceptor": {
                    "lambda": {
                        "arn": LAMBDA_ARN
                    }
                },
                "interceptionPoints": ["REQUEST"],
                "inputConfiguration": {
                    "passRequestHeaders": True  
                }
            }
        ],
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration={
            "customJWTAuthorizer": {
                "discoveryUrl": DISCOVERY_URL,
                "allowedClients": [CLIENT_ID]
            }
        },
        roleArn=GATEWAY_ROLE_ARN
    )
    
    GATEWAY_ID = gateway_response.get('gatewayId')
    print(f"✓ Gateway created: {GATEWAY_ID}")
    
except Exception as e:
    print(f"\n✗ Failed to create Gateway: {e}")
    raise

attaching role policy agentcore-interceptor-gateway-20251126-155213-role
✓ Gateway role created: arn:aws:iam::101546054019:role/agentcore-interceptor-gateway-20251126-155213-role

Creating Gateway with RESPONSE interceptor...
✓ Gateway created: interceptor-gateway-20251126-155213-6nh4f4kgz0


### Step 1.7: Wait for Gateway to be Ready

In [22]:
# Wait for Gateway to be ready using signed requests
print("\nWaiting for Gateway to be ready...")

max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = gateway_client.get_gateway(gatewayIdentifier=GATEWAY_ID)
        status_code = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

        if status_code == 200:
            # gateway_info = response.json()
            status = response.get('status', 'UNKNOWN')
            
            print(f"  [{attempt + 1}/{max_attempts}] Status: {status}")
            
            if status == 'READY':
                GATEWAY_URL = response.get('gatewayUrl')
                print(f"\n✓ Gateway is ready!")
                print(f"  URL: {GATEWAY_URL}")
                
                # Show interceptor configuration
                if 'interceptorConfigurations' in response:
                    interceptor_configs = response['interceptorConfigurations']
                    print(f"\n  Interceptor Configuration:")
                    for idx, config in enumerate(interceptor_configs):
                        print(f"    [{idx}] Interception Points: {config.get('interceptionPoints', [])}")
                        print(f"    [{idx}] Lambda ARN: {config.get('interceptor', {}).get('lambda', {}).get('arn', 'N/A')}")
                        print(f"    [{idx}] Pass Headers: {config.get('inputConfiguration', {}).get('passRequestHeaders', False)}")
                break
            elif status == 'FAILED':
                print(f"\n✗ Gateway creation failed")
                print(f"  Details: {response}")
                raise Exception("Gateway failed")
        else:
            print(f"  [{attempt + 1}/{max_attempts}] HTTP Error: {response.status_code}")
    except Exception as e:
        print(f"  [{attempt + 1}/{max_attempts}] Error: {e}")
    
    time.sleep(10)
else:
    print(f"\n⚠ Timeout waiting for Gateway")
    raise Exception("Gateway timeout")



Waiting for Gateway to be ready...
  [1/30] Status: READY

✓ Gateway is ready!
  URL: https://interceptor-gateway-20251126-155213-6nh4f4kgz0.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp

  Interceptor Configuration:
    [0] Interception Points: ['REQUEST']
    [0] Lambda ARN: arn:aws:lambda:us-east-1:101546054019:function:interceptor-lambda-20251126-155213
    [0] Pass Headers: True


### Step 1.8: Register Sample Inventory Tool with Gateway

Deploy an inventory tool Lambda and register it as a Gateway target.

In [23]:
# Deploy inventory tool Lambda and register as Gateway target
print("="*80)
print("Deploying Inventory Tool Lambda and Registering with Gateway")
print("="*80)

import sys
import importlib

# Import tool module
sys.path.insert(0, str(Path.cwd()))
from src.tools import inventory_tool
inventory_tool = importlib.reload(inventory_tool)

print("✓ Tool module loaded")

# Create IAM role for tool Lambda using utils
print("\nCreating IAM role for tool Lambda...")
TOOL_ROLE_NAME = f"tool-lambda-role-{DEPLOYMENT_ID}"
TOOL_ROLE_ARN = utils.create_lambda_role(
    role_name=TOOL_ROLE_NAME,
    description='Role for inventory tool Lambda function'
)

# Deploy inventory tool Lambda using utils
print("\nDeploying inventory_tool...")

INVENTORY_FUNCTION_NAME = f"inventory-tool-{DEPLOYMENT_ID}"
INVENTORY_LAMBDA_ARN = utils.deploy_lambda_function(
    function_name=INVENTORY_FUNCTION_NAME,
    role_arn=TOOL_ROLE_ARN,
    lambda_code_path='src/tools/inventory_tool.py',
    description='Inventory tool for schema translation demo',
    timeout=30,
    memory_size=256,
    region=REGION,
    environment_vars={'TOOL_NAME': 'inventory_tool'}
)

print(f"    ✓ Created: {INVENTORY_FUNCTION_NAME}")
print(f"    ARN: {INVENTORY_LAMBDA_ARN}")

# Get tool definition
tool_definition = getattr(inventory_tool, 'TOOL_DEFINITION', {
    "name": "inventory_tool",
    "description": "inventory_tool function"
})

deployed_tools = [{
    'tool_name': 'inventory_tool',
    'function_name': INVENTORY_FUNCTION_NAME,
    'lambda_arn': INVENTORY_LAMBDA_ARN,
    'tool_definition': tool_definition
}]

print(f"\n✓ Deployed {len(deployed_tools)} tool Lambda")

# Step 2: Register inventory tool as Gateway target
print(f"\n🎯 Step 2: Registering inventory tool as Gateway target...")
print("-" * 60)

print(f"\n  Registering inventory_tool...")

response = gateway_client.create_gateway_target(
    gatewayIdentifier=GATEWAY_ID,
    name="inventory-tool-target",
    targetConfiguration={
        "mcp": {
            "lambda": {
                "lambdaArn": INVENTORY_LAMBDA_ARN,
                "toolSchema": {
                    "inlinePayload": [tool_definition]
                }
            }
        }
    },
    credentialProviderConfigurations=[
        {"credentialProviderType": "GATEWAY_IAM_ROLE"}
    ]
)

INVENTORY_TARGET_ID = response['targetId']
print(f"    ✓ Target created: {INVENTORY_TARGET_ID}")

# Wait for target to be READY
print(f"    Waiting for target to be READY...")

for attempt in range(18):  # 3 minutes max
    response = gateway_client.get_gateway_target(
        gatewayIdentifier=GATEWAY_ID,
        targetId=INVENTORY_TARGET_ID
    )
    
    status = response.get('status', 'UNKNOWN')
    
    if status == 'READY':
        print(f"    ✓ Target is READY")
        break
    elif status == 'FAILED':
        print(f"    ✗ Target FAILED")
        print(f"    Details: {json.dumps(response, indent=6, default=str)}")
        raise Exception("Target creation failed")
    
    time.sleep(10)
else:
    raise Exception("Timeout waiting for target to be READY")

print(f"\n✅ Inventory tool registered and ready!")

# Store for cleanup
DEPLOYED_TOOL_FUNCTIONS = [INVENTORY_FUNCTION_NAME]
CREATED_TARGET_IDS = [INVENTORY_TARGET_ID]


Deploying Inventory Tool Lambda and Registering with Gateway
✓ Tool module loaded

Creating IAM role for tool Lambda...
✓ IAM role created: tool-lambda-role-20251126-155213

Deploying inventory_tool...
✓ Lambda created: inventory-tool-20251126-155213
    ✓ Created: inventory-tool-20251126-155213
    ARN: arn:aws:lambda:us-east-1:101546054019:function:inventory-tool-20251126-155213

✓ Deployed 1 tool Lambda

🎯 Step 2: Registering inventory tool as Gateway target...
------------------------------------------------------------

  Registering inventory_tool...
    ✓ Target created: Q24PSAJ7GQ
    Waiting for target to be READY...
    ✓ Target is READY

✅ Inventory tool registered and ready!


---

## Part 2: Testing

### Step 2.1: List All Tools on the Gateway

In [24]:
import requests
import json


def fetch_access_token(client_id, client_secret, token_url):
  response = requests.post(
    token_url,
    data="grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}".format(client_id=client_id, client_secret=client_secret),
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
  )

  return response.json()['access_token']

def list_tools(gateway_url, access_token):
  headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {access_token}"
  }

  payload = {
      "jsonrpc": "2.0",
      "id": "list-tools-request",
      "method": "tools/list"
  }

  response = requests.post(gateway_url, headers=headers, json=payload)
  return response.json()

access_token = fetch_access_token(CLIENT_ID, CLIENT_SECRET, TOKEN_URL)
tools = list_tools(GATEWAY_URL, access_token)
print(json.dumps(tools, indent=2))

{
  "jsonrpc": "2.0",
  "id": "list-tools-request",
  "result": {
    "tools": [
      {
        "inputSchema": {
          "type": "object",
          "properties": {
            "itemId": {
              "description": "The unique item identifier (e.g., 'ITEM-12345')",
              "type": "string"
            },
            "location": {
              "description": "The warehouse location (e.g., 'East Warehouse'). If not provided, a random location will be selected.",
              "type": "string"
            }
          },
          "required": [
            "itemId"
          ]
        },
        "name": "inventory-tool-target___inventory_tool",
        "description": "Retrieve inventory information by Item ID. Returns data in camelCase format."
      }
    ]
  }
}


### Step 2.2: Invoke Inventory Tool with Arguments

- When tool is invoked using "item_id" argument, the lambda request interceptor changes the schema from "item_id" to "itemId" for fitting the tool requirements
- Also the request interceptor adds in an additional argument "location" as "East Warehouse" by default if location is not provided

In [28]:
# Test Schema Translation by invoking the tool
import requests

print("Testing Schema Translation interceptor...")
print(f"Gateway URL: {GATEWAY_URL}")

# Get OAuth token
token_data = utils.get_token(
    user_pool_id=USER_POOL_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope_string="gateway/tools",
    REGION=REGION
)

if 'error' in token_data:
    print(f"✗ Token request failed: {token_data['error']}")
else:
    token = token_data['access_token']
    print(f"✓ Token obtained")
    
    # Call the customer data tool
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 1,
        "params": {
            "name": "inventory-tool-target___inventory_tool",
            "arguments": {"item_id": "ITEM-12345"}
        }
    }
    
    response = requests.post(
        GATEWAY_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=mcp_request
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"\n✓ Tool invoked successfully")
        print(f"\nSchema translation happened by changing item_id to itemId and adding default location as East Warehouse")
        print(json.dumps(result, indent=2))
    else:
        print(f"✗ Request failed: {response.status_code}")
        print(f"Response: {response.text}")

Testing Schema Translation interceptor...
Gateway URL: https://interceptor-gateway-20251126-155213-6nh4f4kgz0.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
6ff968ucqm2f5a5rbqejg8pobg
✓ Token obtained

✓ Tool invoked successfully

Schema translation happened by changing item_id to itemId and adding default location as East Warehouse
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "isError": false,
    "content": [
      {
        "type": "text",
        "text": "{\"statusCode\":200,\"body\":\"{\\\"tool\\\": \\\"inventory_tool\\\", \\\"result\\\": {\\\"itemId\\\": \\\"ITEM-12345\\\", \\\"productName\\\": \\\"Bluetooth Speaker\\\", \\\"quantityAvailable\\\": 138, \\\"warehouseLocation\\\": \\\"East Warehouse\\\", \\\"lastUpdated\\\": \\\"2025-10-31T00:04:49.261206\\\", \\\"isInStock\\\": true, \\\"reorderLevel\\\": 63, \\\"supplierInfo\\\": {\\\"supplierId\\\": \\\"SUP-4215\\\", \\\"supplierName\\\": \\\"Supplier D\\\", \\\"leadTimeInDays\\\": 5}, \\\"productDetails\\\": {\\\"w

### Step 2.3: Invoke tool with multiple arguments

- When tool is invoked using "item_id" argument, the lambda request interceptor changes the schema from "item_id" to "itemId"
- since location value is provided by the user, this time it will be passed to the tool and the default value wont be used

In [30]:
# Test Schema Translation by invoking the tool
import requests

print("Testing Schema Translation interceptor...")
print(f"Gateway URL: {GATEWAY_URL}")

# Get OAuth token
token_data = utils.get_token(
    user_pool_id=USER_POOL_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    scope_string="gateway/tools",
    REGION=REGION
)

if 'error' in token_data:
    print(f"✗ Token request failed: {token_data['error']}")
else:
    token = token_data['access_token']
    print(f"✓ Token obtained")
    
    # Call the customer data tool
    mcp_request = {
        "jsonrpc": "2.0",
        "method": "tools/call",
        "id": 1,
        "params": {
            "name": "inventory-tool-target___inventory_tool",
            "arguments": {
                    "item_id": "ITEM-12345",
                    "location": "West Warehouse"
            }
        }
    }
    
    response = requests.post(
        GATEWAY_URL,
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=mcp_request
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"\n✓ Tool invoked successfully")
        print(f"\nSchema translation happened by changing item_id to itemId and overriding with specified location")
        print(json.dumps(result, indent=2))
    else:
        print(f"✗ Request failed: {response.status_code}")
        print(f"Response: {response.text}")

Testing Schema Translation interceptor...
Gateway URL: https://interceptor-gateway-20251126-155213-6nh4f4kgz0.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
6ff968ucqm2f5a5rbqejg8pobg
✓ Token obtained

✓ Tool invoked successfully

Schema translation happened by changing item_id to itemId and overriding with specified location
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "isError": false,
    "content": [
      {
        "type": "text",
        "text": "{\"statusCode\":200,\"body\":\"{\\\"tool\\\": \\\"inventory_tool\\\", \\\"result\\\": {\\\"itemId\\\": \\\"ITEM-12345\\\", \\\"productName\\\": \\\"Premium Laptop\\\", \\\"quantityAvailable\\\": 776, \\\"warehouseLocation\\\": \\\"West Warehouse\\\", \\\"lastUpdated\\\": \\\"2025-11-17T00:07:16.140124\\\", \\\"isInStock\\\": false, \\\"reorderLevel\\\": 33, \\\"supplierInfo\\\": {\\\"supplierId\\\": \\\"SUP-2180\\\", \\\"supplierName\\\": \\\"Supplier B\\\", \\\"leadTimeInDays\\\": 19}, \\\"productDetails\\\": {\\\"weightInK

---

# Part 3: Cleanup - Delete All Resources

⚠️ **WARNING: This will DELETE all resources created in Part 1!**

Only run this section if you want to clean up everything.

### Step 3.1: Delete Created Resources

In [31]:
# Cleanup - Delete all created resources using utils
print("Starting cleanup...")

# 1. Delete gateway targets
if 'CREATED_TARGET_IDS' in globals() and 'GATEWAY_ID' in globals():
    utils.delete_gateway_targets(gateway_client, GATEWAY_ID, CREATED_TARGET_IDS)
    # Wait for target deletions to complete before deleting gateway
    time.sleep(5)

# 2. Delete gateway
if 'GATEWAY_ID' in globals():
    utils.delete_gateway(gateway_client, GATEWAY_ID)
    print("✓ Deleted gateway")

# 3. Delete Lambda functions (tools + interceptor)
lambda_functions_to_delete = []
if 'DEPLOYED_TOOL_FUNCTIONS' in globals():
    lambda_functions_to_delete.extend(DEPLOYED_TOOL_FUNCTIONS)
if 'REQUEST_LAMBDA_FUNCTION_NAME' in globals():
    lambda_functions_to_delete.append(LAMBDA_FUNCTION_NAME)

if lambda_functions_to_delete:
    utils.delete_lambda_functions(lambda_functions_to_delete, REGION)

# 4. Delete IAM roles
if 'LAMBDA_ROLE_NAME' in globals():
    utils.delete_iam_role(LAMBDA_ROLE_NAME)
if 'DEPLOYMENT_ID' in globals():
    utils.delete_iam_role(f"tool-lambda-role-{DEPLOYMENT_ID}")
    utils.delete_iam_role(f"agentcore-{GATEWAY_NAME}-role")

# 5. Delete Cognito user pool
if 'USER_POOL_ID' in globals():
    utils.delete_cognito_user_pool(USER_POOL_ID, REGION)

print("\n✓ Cleanup complete!")



Starting cleanup...
Deleting 1 gateway targets...
  ✓ Deleted target: Q24PSAJ7GQ
Deleting all targets for gateway interceptor-gateway-20251126-155213-6nh4f4kgz0
Deleting gateway  interceptor-gateway-20251126-155213-6nh4f4kgz0
✓ Deleted gateway
Deleting 2 Lambda functions...
  ✓ Deleted Lambda: inventory-tool-20251126-155213
  ✓ Deleted Lambda: interceptor-lambda-20251126-155213
✓ Deleted IAM role: interceptor-lambda-role-20251126-155213
✓ Deleted IAM role: tool-lambda-role-20251126-155213
✓ Deleted IAM role: agentcore-interceptor-gateway-20251126-155213-role
✗ Failed to delete user pool: An error occurred (InvalidParameterException) when calling the DeleteUserPool operation: User pool cannot be deleted. It has a domain configured that should be deleted first.

✓ Cleanup complete!


---

# Summary

This notebook demonstrates schema translation using Request Lambda interceptor:

1. ✅ **Setup** - Created Lambda interceptor, IAM roles, Cognito, and Gateway
2. ✅ **Test** - Verified Schema Translation through Gateway responses
3. ✅ **Cleanup** - Deleted all resources

## What We Demonstrated

- **Lambda REQUEST interceptor** that modifies Gateway requests
- **Transforming Schema and adding additional parameters** through the request chain
- **Gateway integration** with custom interceptors
- **Complete resource lifecycle** management

## Next Steps
- Customize schema translation logic based on your use case
- Adding more sophisticated schema translation requirements
- Integrate with compliance logging
- Monitor CloudWatch logs for debugging